# Cross-sell: target the right customer

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer
#from hyperopt import hp, tpe
#from hyperopt.fmin import fmin
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read train and test data
train = pd.read_csv('train.csv')
test = pd.read_csv('test_plBmD8c.csv')
test['RESPONDERS'] = np.nan

In [ ]:
combin = pd.concat([train,test])

In [ ]:
# Cleaning the date variables
from datetime import date
date_cols = ['MATURITY_GL', 'MATURITY_LAP', 'MATURITY_LAS','CLOSED_DATE']

for i in date_cols:
    combin[i] = pd.to_datetime(combin[i],format='%d%b%Y')
    if i == 'CLOSED_DATE':
        d0 = date(2017,10,10)
    else:
        d0 = date(2025, 1, 1)
    combin[i] = (d0 - combin[i]).dt.days

The next step is to create some new features aggregatin the similar paramaters/ taking the median in a few cases

In [ ]:
amb_cols = [col for col in combin.columns if 'AMB_MON' in col]
combin['Median_AMB'] = combin[amb_cols].median(axis = 1)

In [ ]:
eop_cols = [col for col in combin.columns if 'EOP' in col]
combin['Median_EOP'] = combin[eop_cols].median(axis = 1)

In [ ]:
d_amt_cols = [col for col in combin.columns if 'D_AMT_L3' in col]
c_amt_cols = [col for col in combin.columns if 'C_AMT_L3' in col]
Total_Debit = combin[d_amt_cols].sum(axis = 1)
Total_Credit = combin[c_amt_cols].sum(axis = 1)
combin['Net Balance'] = Total_Credit - Total_Debit

In [ ]:
max_c_cols = [col for col in combin.columns if 'MAX_C' in col]
combin['Median_max_c'] = combin[max_c_cols].median(axis = 1)

In [ ]:
sal_cols = [col for col in combin.columns if 'SAL_MON' in col]
combin['Median_sal'] = combin[sal_cols].median(axis = 1)

In [ ]:
d_count_cols = [col for col in combin.columns if 'D_COUNT' in col]
combin['Median_d_count'] = combin[d_count_cols].median(axis = 1)

In [ ]:
combin['N_CASA_MAX-MIN'] = combin['N_CASA_MAX_BALANCE_MTD'] - combin['N_CASA_MIN_BALANCE_MTD']

In [ ]:
cc_spend_cols = ['CC_SPEND_MON_04', 'CC_SPEND_MON_01', 'CC_SPEND_MON_02', 'CC_SPEND_MON_03', 'CC_SPEND_MON_06', 'CC_SPEND_MON_05',]
dc_spend_cols = ['DC_SPEND_MON_04', 'DC_SPEND_MON_01', 'DC_SPEND_MON_02', 'DC_SPEND_MON_03', 'DC_SPEND_MON_06', 'DC_SPEND_MON_05',]
combin['Total_CC_SPEND'] = combin[cc_spend_cols].sum(axis = 1)
combin['Total_DC_SPEND'] = combin[dc_spend_cols].sum(axis = 1)

ent_cols = ['DC_SPEND_MON_04_ENT',
            'DC_SPEND_MON_03_ENT',
            'DC_SPEND_MON_02_ENT',
            'DC_SPEND_MON_01_ENT',
            'DC_SPEND_MON_05_ENT',
            'CC_SPEND_MON_04_ENT',
            'CC_SPEND_MON_02_ENT',
            'CC_SPEND_MON_01_ENT',
            'CC_SPEND_MON_03_ENT']
med_cols = ['DC_SPEND_MON_04_MED',
            'DC_SPEND_MON_03_MED',
            'DC_SPEND_MON_02_MED',
            'DC_SPEND_MON_01_MED',
            'DC_SPEND_MON_05_MED',
            'CC_SPEND_MON_04_MED',
            'CC_SPEND_MON_02_MED',
            'CC_SPEND_MON_01_MED',
            'CC_SPEND_MON_03_MED']
hmd_cols = [col for col in combin.columns if 'HMD' in col]
hby_cols = [col for col in combin.columns if 'HBY' in col]
care_cols = ['DC_SPEND_MON_04_CARE',
             'DC_SPEND_MON_03_CARE',
             'DC_SPEND_MON_02_CARE',
             'DC_SPEND_MON_01_CARE',
             'DC_SPEND_MON_05_CARE',
             'CC_SPEND_MON_04_CARE',
             'CC_SPEND_MON_02_CARE',
             'CC_SPEND_MON_01_CARE',
             'CC_SPEND_MON_03_CARE']
trl_cols = [col for col in combin.columns if 'TRL' in col]
rst_cols = [col for col in combin.columns if 'RST' in col]
jer_cols = [col for col in combin.columns if 'JER' in col]
htl_cols = [col for col in combin.columns if 'HTL' in col]
atm_cols = [col for col in combin.columns if 'ATM' in col]
combin['TOTAL_ENT'] = combin[ent_cols].sum(axis = 1)
combin['TOTAL_MED'] = combin[med_cols].sum(axis = 1)
combin['TOTAL_HMD'] = combin[hmd_cols].sum(axis = 1)
combin['TOTAL_HBY'] = combin[hby_cols].sum(axis = 1)
combin['TOTAL_CARE'] = combin[care_cols].sum(axis = 1)
combin['TOTAL_TRL'] = combin[trl_cols].sum(axis = 1)
combin['TOTAL_RST'] = combin[rst_cols].sum(axis = 1)
combin['TOTAL_JER'] = combin[jer_cols].sum(axis = 1)
combin['TOTAL_HTL'] = combin[htl_cols].sum(axis = 1)
combin['TOTAL_ATM'] = combin[atm_cols].sum(axis = 1)


In [ ]:
live_loan_tags = [col for col in combin.columns if 'TAG' in col and 'TOP_CORP' not in col and 'PA_PQ' not in col]
closed_loan_tags = [col for col in combin.columns if '_CLOSED' in col and 'SCRUB' not in col and 'MON' not in col]
combin['LIVE_LOAN_TAGS_SUM'] = combin[live_loan_tags].sum(axis = 1)
combin['CLOSED_LOAN_TAGS_SUM'] = combin[closed_loan_tags].sum(axis = 1)


In [ ]:
page_visit_cols =['TRAVEL_MON_01',
                  'CDL_MON_03',
                  'GL_MON_03',
                  'TWL_MON_03',
                  'PL_MON_03',
                  'CDL_MON_02',
                  'GL_MON_02',
                  'TWL_MON_02',
                  'PL_MON_02',
                  'CDL_MON_01',
                  'GL_MON_01',
                  'TWL_MON_01',
                  'PL_MON_01']

combin['TOTAL_PAGE_VISITS'] = combin[page_visit_cols].sum(axis = 1)

In [ ]:
stmt_cols = ['STMT_MON_01','STMT_MON_02','STMT_MON_03']
combin[stmt_cols].fillna(0,inplace = True)
combin.fillna(-1,inplace = True)
combin.replace('Y', 1, inplace = True)
combin.replace('N', 0, inplace = True)

In [ ]:
#Processing zip codes to create a meaningful feature
combin['new_zip'] = combin['ZIP_CODE_FINAL'].astype(str).str[0:3]
counts = combin.new_zip.value_counts()
combin.new_zip[combin['new_zip'].isin(counts[counts > 9000].index)] = 2
combin.new_zip[combin['new_zip'].isin(counts[counts > 1000].index) & combin['new_zip'].isin(counts[counts < 9000].index)] = 1
combin.new_zip[combin['new_zip'].isin(counts[counts < 1000].index)] = 0
combin.new_zip[combin['ZIP_CODE_FINAL'] == -1] = -1

In [ ]:
combin.loc[combin['PA_PQ_TAG'].str.contains("INCOME", na = False),'PA_PQ_TAG'] = combin.PA_PQ_TAG[combin['PA_PQ_TAG'].str.contains("INCOME", na = False)].apply(lambda x: x.split('INCOME_')[1]).apply(lambda x: x.split('K')[0])


In [ ]:
combin.loc[combin['PA_PQ_TAG'] == 'Green Channel','PA_PQ_TAG'] = 1
combin.loc[combin['PA_PQ_TAG'] == 'PQ with Ref dedupe','PA_PQ_TAG'] = 2
combin.loc[combin['PA_PQ_TAG'] == 'PQ with Dedupe','PA_PQ_TAG'] = 2
combin.loc[combin['PA_PQ_TAG'] == 'PQ with Neg dedupe','PA_PQ_TAG'] = 3
combin.loc[combin['PA_PQ_TAG'] == 'PQ with Account','PA_PQ_TAG'] = 4
combin.loc[combin['PA_PQ_TAG'] == 'PQ with KYC','PA_PQ_TAG'] = 5
combin.loc[combin['PA_PQ_TAG'] == 'PQ with Coapp','PA_PQ_TAG'] = 6
combin.loc[combin['PA_PQ_TAG'] == 'STP','PA_PQ_TAG'] = 7

In [ ]:
dict_act_type = {'SA': 0, 'CSA': 1}

dict_gender = {'M': 0, 'F': 1}

dict_legal_entity = {'INDIVIDUAL': 0, 'OTHERS':1, 'NON_GOVERNMENTAL_ORG':1, 'ASSOCIATION':1,
                     'FIRM':1, 'PROPRIETERSHIP':1, 'TRUSTS':1}

dict_channel_click_disp = {'NONE':0, 'DISP':1, 'CLICK':2}

dict_designation_final = {'MISSING':-1, 'ENGINEER':1, 'MANAGER':2, 'EXECUTIVE':1, 'SUP/TECH/OPR':1,
                          'OFFICER':2, 'OTHERS':0, 'DEVEP/PROG/ANALYST/ASSO':1, 'REMAING':0,
                          'TOP_MANAG/DOC':3, 'TEACH/PROF/CONSULT':2}

dict_neft_tpt_imps_cat = {'NO_TPT_TXN':0, 'NOT_RELEVANT':-1, 'TRANSFER':1, 'LOAN':2, 'PERSONAL':3,
       'RETURN':4, 'RENT':5, 'TRAVEL':6, 'CC_PAYMENT':7, 'EMI':8, 'CHILD_BIR':9,
       'EDUCATION':10, 'GAS':11, 'ADVANCE':12, 'MEDICAL':13, 'HELP':14, 'ELECTRICI':15,
       'WEDDING':16, 'BUILDER':17, 'OPERATION':18, 'JEWELLERY':19, 'NO_NEFT_TXN': 0, 'NO_IMPS_TXN': 0}

dict_top_corp_tag = {'NONE':-1, 'TOP_800':1, 'TOP_100':2}

combin.replace({"ACT_TYPE": dict_act_type}, inplace=True)
combin.replace({"GENDER": dict_gender}, inplace=True)
combin.replace({"LEGAL_ENTITY": dict_legal_entity}, inplace=True)
combin.replace({'CHANNEL_CLICK_DISP': dict_channel_click_disp}, inplace=True)
combin.replace({'DESIGNATION_FINAL': dict_designation_final}, inplace=True)
combin.replace({'NEFT_DC_CATEGORY': dict_neft_tpt_imps_cat}, inplace=True)
combin.replace({'NEFT_CC_CATEGORY': dict_neft_tpt_imps_cat}, inplace=True)
combin.replace({'TPT_DC_CATEGORY_MON_01': dict_neft_tpt_imps_cat}, inplace=True)
combin.replace({'TPT_CC_CATEGORY_MON_01': dict_neft_tpt_imps_cat}, inplace=True)
combin.replace({'IMPS_CC_CATEGORY_MON_01': dict_neft_tpt_imps_cat}, inplace=True)
combin.replace({'TOP_CORP_TAG': dict_top_corp_tag}, inplace=True)


In [ ]:
y_all = combin.RESPONDERS.iloc[0:300000]
combin.drop(['RESPONDERS','CUSTOMER_ID','ZIP_CODE_FINAL','OCCUP_ALL_NEW'],axis = 1,inplace=True)
test_new = combin.iloc[300000:,:]
train_new = combin.iloc[0:300000,:]

In [ ]:
# Remove features with zero variance
selector = VarianceThreshold(threshold=0)
selector.fit(train_new)
z = selector.get_support(indices=False)
zz = pd.DataFrame({'Feature':train_new.columns.values, 'ret': z})
cols = np.array(zz.loc[zz.ret == 1,'Feature'])
train_new = pd.DataFrame(selector.fit_transform(train_new),columns=cols)
test_new = pd.DataFrame(selector.transform(test_new),columns=cols)

## XGBoost

In [ ]:
params = {}
params['objective'] = 'binary:logistic'
params['eta'] = 0.01
params['booster'] = 'gbtree'
params['silent'] = True
params['max_depth'] = 7
params['subsample'] = 0.9
params['colsample_bytree'] = 0.65
params['tree_method'] = 'exact'
params['eval_metric'] = 'auc'
#params['scale_pos_weight'] = 2
#params['gamma'] = 0.076
params['gamma'] = 0.1
params['min_child_weight'] = 4

In [ ]:
kfold = 10
X = train_new.values
y = y_all.values
sub_xgb=test['CUSTOMER_ID'].to_frame()
sub_xgb['RESPONDERS']=0
nrounds = 3000

In [ ]:
# Take average of different xgboost models for better generalization
features = train_new.columns
skf = StratifiedKFold(n_splits=kfold, random_state=0, shuffle=True)
np.random.seed(0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100,
                          maximize=True, verbose_eval=100)
    sub_xgb['RESPONDERS'] += xgb_model.predict(xgb.DMatrix(test_new[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
sub_xgb.to_csv('xgb_tuned_averaged.csv', index=False, float_format='%.5f')

## Light Gradient Boosting Machine

In [ ]:
# lgb
kfold = 5
import lightgbm as lgb
def lift_lgb(pred, dtrain):
    y = list(dtrain.get_label())
    return 'max_lift', lift(y, pred), True

sub_lgb=test['CUSTOMER_ID'].to_frame()
sub_lgb['RESPONDERS']=0

params = {'metric': 'auc', 'learning_rate' : 0.01, 'max_depth':9, 'max_bin':20,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,'bagging_freq':10,  'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=5, shuffle = True)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=100, 
                  early_stopping_rounds=100)
    sub_lgb['RESPONDERS'] += lgb_model.predict(test_new[features].values, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
sub_lgb.to_csv('lgb_tuned_averaged.csv', index=False, float_format='%.5f')

In [ ]:
# Calculating Normalized rank and making ensemble
xgb_pred = pd.read_csv('xgb_tuned_averaged.csv')
lgb_pred = pd.read_csv('lgb_tuned_averaged.csv')
xgb_pred['rank/length'] = xgb_pred['RESPONDERS'].rank(ascending = 1)/xgb_pred.shape[0]
lgb_pred['rank/length'] = lgb_pred['RESPONDERS'].rank(ascending = 1)/lgb_pred.shape[0]
sub = pd.DataFrame({'CUSTOMER_ID': test.CUSTOMER_ID, 'RESPONDERS':0.5 * xgb_pred['rank/length'] + 0.5 * lgb_pred['rank/length']})
sub.to_csv('sub_ens_rank_norm.csv',index=False,float_format='%.6f')